# Simple example notebook that causes the largest OOI.pangeo.oi server to run out of memory when trying to lazily load and concatenate AMPS netcdfs



In [1]:
import xarray as xr
import fsspec
import gcsfs
from tqdm import tqdm
xr.set_options(display_style="html");

### list the netcdf files

In [12]:
fs = gcsfs.GCSFileSystem(project='ldeo-glaciology', mode='ab', cache_timeout = 0)
NCs = fs.glob('gs://ldeo-glaciology/AMPS/WRF_24/domain_03/wrf-2016120*')
print(len(NCs))
print(NCs[0])

8
ldeo-glaciology/AMPS/WRF_24/domain_03/wrf-2016122500_f003-cf.nc


### Loop through all the netcdf files in the AMPS directory and concat each one.
This cell causes the kernel to restart when you try to loop over all the netcdfs (len (NCs) = 55) we currently have in GCS.



In [13]:
## load the first file to inialize the xarray
url = 'gs://' + NCs[0]
with  fsspec.open(url, mode='rb')  as openfile:  
    AMPS = xr.open_dataset(openfile, chunks={})  # these chunk sizes produce chunks of reasonable data volumes and which stretch through all time

## load the other files, each time concaternating them onto an xarray (AMPS) that grows in the time dimension each iteration. 
for i in tqdm(range(1, len(NCs)-1)):  
    url = 'gs://' + NCs[i]
    with  fsspec.open(url, mode='rb')  as openfile:  
        temp = xr.open_dataset(openfile, chunks={})  # these chunk sizes produce chunks of reasonable data volumes and which stretch through all time
    AMPS = xr.concat([AMPS,temp],'time')

100%|██████████| 6/6 [00:58<00:00,  9.80s/it]


I am interested to know why the cell above fills the memory, when my intention was to only be loading things lazily. Note that when I load a smaller number of files (e.g., 45), instead of the full 55, I produce an xarray which seems to be made up of dask arrays as intended, so I dont why does it take up so much space on disk? Is it something to do with the chunking? increasing chunk size with AMPS.chunk doesnt seems to change the memory usage (monitored with top). 

In [14]:
AMPS

<xarray.Dataset>
Dimensions:   (pressure: 5, south_north: 825, time: 7, west_east: 537)
Coordinates:
  * time      (time) datetime64[ns] 2016-12-25T03:00:00 ... 2016-12-25T21:00:00
  * pressure  (pressure) float32 1000.0 850.0 700.0 500.0 300.0
    lat       (south_north, west_east) float32 -80.58021 ... -67.988365
    lon       (south_north, west_east) float32 59.897705 59.80359 ... -157.98875
Dimensions without coordinates: south_north, west_east
Data variables:
    DateTime  (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    year      (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    month     (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    day       (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    hour      (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    minute    (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    Z_sfc     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    SST       (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    T_sfc     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    p_sfc     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    slp       (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    T_2m      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    Td_2m     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    r_v_2m    (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    q_2m      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    rh_2m     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    u_10m_tr  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    v_10m_tr  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    ws_10m    (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    wd_10m    (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    precip_g  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    Z_p       (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    T_p       (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    Td_p      (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    r_v_p     (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    q_p       (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    SW_d      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LW_d      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LW_u      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LW_d_acc  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LW_u_acc  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    albedo    (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    SH        (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LH        (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    SH_acc    (time, south_north, west_east) float32 dask.array<chunksiz